# Iniciando Sessão Spark

In [1]:
from pyspark.sql import SparkSession

# Iniciar a sessão Spark com a configuração do Delta Lake
spark = (SparkSession.builder
    .appName("DeltaLake")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")  # Extensão do Delta
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")  # Catálogo do Delta
    .config("spark.jars", "/usr/local/spark/jars/delta-spark_2.12-3.0.0.jar,/usr/local/spark/jars/delta-storage-3.0.0.jar")  # Caminho dos JARs no Docker
    .getOrCreate()) # Se utilizar as limitações de hardware pelo spark, mover para o final
#    .config("spark.executor.cores", "2")  # Limitar o número de núcleos por executor
#    .config("spark.driver.cores", "1")    # Limitar o número de núcleos para o driver
#    .config("spark.executor.memory", "4g")
#    .config("spark.driver.memory", "4g")
#    .config("spark.memory.fraction", "0.3")  # Ajuste para controlar a fração de memória usada pelo Spark
#    .config("spark.memory.storageFraction", "0.2")  # Fração da memória usada para armazenar dados persistentes"""

from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import udf

import time

# Verificar se a sessão Spark foi criada com sucesso
print(spark.version)

3.5.0


# Importando CSV

In [ ]:
df_test = spark.read.csv("/home/jovyan/app/data/raw/temas_ambientais.csv", sep=";", header=True)
df_test.show()

# Salvando Bronze (Distribuída)

In [ ]:
df_test = df_test.dropDuplicates()

In [ ]:
df_test.write.mode("overwrite").format("delta").save("/home/jovyan/app/data/bronze/bronze_distribuida")

# Limpeza e Salvando Silver (Distribuída)

In [ ]:
df_test = spark.read.format("delta").load("/home/jovyan/app/data/bronze/bronze_distribuida")
df_test.show(10)

In [ ]:
# Criar uma lista com o nome das colunas e o número de nulos em cada uma
null_counts_1 = df_test.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_test.columns]).first().asDict()

# Exibir o resultado
null_counts_1

In [ ]:
df_test.printSchema()

In [ ]:
int_columns = ["codigo_ibge"]
float_columns = ["area_do_imovel", "area_liquida", "area_remanescente_vegetacao_nativa", 
                 "area_reserva_legal_proposta", "area_preservacao_permanente", "area_nao_classificada",
                 "latitude", "longitude", "area_rural_consolidada", "area_servidao_administrativa",
                 "modulos_fiscais", "area_uso_restrito", "area_reserva_legal_averbada",
                 "area_reserva_legal_aprovada_nao_averbada", "area_pousio"]
bool_columns = ["solicitacao_adesao_pra"]
date_columns = ["data_inscricao", "data_alteracao_condicao_cadastro", "data_ultima_retificacao"]
text_column = [x for x in df_test.columns if x not in int_columns+float_columns+bool_columns+date_columns]

In [ ]:
# Dados Para Integer
df_test = df_test.withColumn("codigo_ibge", col("codigo_ibge").cast(IntegerType()))

# Dados Para Float
for i in float_columns:
    df_test = df_test.withColumn(i, col(i).cast(FloatType()))

# Dados para Bool
df_test = df_test.withColumn("solicitacao_adesao_pra", 
                             when(col("solicitacao_adesao_pra") == "Sim", True).
                             when(col("solicitacao_adesao_pra") == "Nao", False).
                             otherwise(None))

# Dados para Date
for i in date_columns:
    df_test = df_test.withColumn(i, to_timestamp(regexp_replace(col(i), r"\.\d+$", ""), "yyyy-MM-dd HH:mm:ss"))

df_test.printSchema()

In [ ]:
# Criar uma lista com o nome das colunas e o número de nulos em cada uma
null_counts_2 = df_test.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_test.columns]).first().asDict()

# Exibir o resultado
null_counts_2

In [ ]:
if null_counts_1 == null_counts_2:
    print("Os valores de nulos são os mesmos em ambas as versões.")
else:
    print("Há diferenças nos valores de nulos.")
    # Mostrar as diferenças
    for coluna in null_counts_1.keys():
        if null_counts_1[coluna] != null_counts_2[coluna]:
            print(f"Coluna '{coluna}': null_counts_1 = {null_counts_1[coluna]}, null_counts_2 = {null_counts_2[coluna]}")

# Salvando Silver (Distribuída - Por Estado)

In [ ]:
(df_test.write.mode("overwrite")
              .format("delta")
              .partitionBy("uf")
              .save("/home/jovyan/app/data/silver/silver_distribuida_uf"))

# Querys a partir das Delta Tables

## Funções e Definições

In [2]:
def get_time_delta(query, path):
    start_time = time.time()

    table = spark.sql(query)

    (table.write.mode("overwrite")
                .format("delta")
                .save(path))
    
    spark.sql(f"""SELECT * FROM delta.`{path}`""").show()

    end_time = time.time()

    tempo_query = end_time - start_time

    return tempo_query

def is_point_in_path(x: float, y: float, poly: list[tuple[float, float]]) -> bool:
    """Determine if the point is on the path, corner, or boundary of the polygon

    Args:
    x -- The x coordinates of point.
    y -- The y coordinates of point.
    poly -- a list of tuples [(x, y), (x, y), ...]

    Returns:
        True if the point is in the path or is a corner or on the boundary"""
    
    c = False

    for i in range(len(poly)):
        ax, ay = poly[i]
        bx, by = poly[i - 1]
        if (x == ax) and (y == ay):
            # point is a corner
            return True
        if (ay > y) != (by > y):
            slope = (x - ax) * (by - ay) - (bx - ax) * (y - ay)
            if slope == 0:
                # point is on boundary
                return True
            if (slope < 0) != (by < ay):
                c = not c

    return c

# Registrar a UDF no Spark
poly = [(-53.5325072, -19.4632582), (-51.0495971, -19.1625841), (-51.3734501, -16.1924262), (-53.8181518, -16.4010783), (-53.5325072, -19.4632582)]

is_point_in_polygon_udf = udf(lambda x, y: is_point_in_path(x, y, poly), BooleanType())

spark.udf.register("is_point_in_polygon", is_point_in_polygon_udf)

## Consulta 1

Recupere a soma de área (em hectares) para todas as propriedades agrícolas que pertencem ao MS e MT. Ordene os resultados em ordem decrescente.

In [3]:
query = """
SELECT 
   uf,
   FORMAT_NUMBER(SUM(area_do_imovel), 3) as soma_area_hectares
FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`

WHERE uf IN ('MS', 'MT')

GROUP BY uf

ORDER BY soma_area_hectares DESC

"""

tempo_delta_1 = get_time_delta(query, "/home/jovyan/app/data/gold/delta/query_1")
tempo_delta_1

+---+------------------+
| uf|soma_area_hectares|
+---+------------------+
| MT|    84,587,089.218|
| MS|    36,321,882.660|
+---+------------------+



23.51868510246277

## Consulta 2
Filtre todas as propriedades que pertecem a região sudeste. 

In [4]:
query = """
SELECT
   uf,
   codigo_ibge

FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`

"""

tempo_delta_2 = get_time_delta(query, "/home/jovyan/app/data/gold/delta/query_2")
tempo_delta_2

+---+-----------+
| uf|codigo_ibge|
+---+-----------+
| AC|    1200807|
| AC|    1200344|
| AC|    1200104|
| AC|    1200401|
| AC|    1200302|
| AC|    1200708|
| AC|    1200302|
| AC|    1200450|
| AC|    1200179|
| AC|    1200104|
| AC|    1200401|
| AC|    1200104|
| AC|    1200344|
| AC|    1200500|
| AC|    1200500|
| AC|    1200708|
| AC|    1200450|
| AC|    1200609|
| AC|    1200138|
| AC|    1200609|
+---+-----------+
only showing top 20 rows



14.173457622528076

## Consulta 3
Recupere todas as propriedades rurais que estão localizadas dentro de uma área geográfica específica delimitada por um polígono. Este polígono é descrito pelas seguintes coordenadas: POLYGON ((-53.5325072 -19.4632582, -51.0495971 -19.1625841, -51.3734501 -16.1924262, -53.8181518 -16.4010783, -53.5325072 -19.4632582))
Verificar se um centroide está dentro dos pontos informados (ou mais).

In [5]:
query = """
SELECT
    codigo_ibge,
    latitude,
    longitude,
    is_point_in_polygon(latitude, longitude) AS inside_polygon

FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`

WHERE is_point_in_polygon(latitude, longitude) = True
"""

tempo_delta_3 = get_time_delta(query, "/home/jovyan/app/data/gold/delta/query_3")
tempo_delta_3

+-----------+--------+---------+--------------+
|codigo_ibge|latitude|longitude|inside_polygon|
+-----------+--------+---------+--------------+
+-----------+--------+---------+--------------+



21.466993808746338

## Consulta 4
Calcule quantas propriedades foram cadastradas por ano. Apresente os resultados em ordem cronológica.

In [6]:
query = """
SELECT
   YEAR(data_inscricao) AS ano_inscricao,
   COUNT(*) AS qnt_ano

FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`

GROUP BY ano_inscricao

ORDER BY ano_inscricao
"""

tempo_delta_4 = get_time_delta(query, "/home/jovyan/app/data/gold/delta/query_4")
tempo_delta_4

+-------------+-------+
|ano_inscricao|qnt_ano|
+-------------+-------+
|         NULL|     12|
|         2013|    743|
|         2014| 227468|
|         2015|1240176|
|         2016|1828786|
|         2017| 794777|
|         2018| 724856|
|         2019| 746570|
|         2020| 463322|
|         2021| 443942|
|         2022| 368448|
+-------------+-------+



14.781372547149658

## Consulta 5
Calcule o percentual médio de área remanescente de vegetação nativa em comparação a área total da propriedade

In [7]:
query = """
SELECT
   codigo_ibge,
   area_do_imovel,
   area_remanescente_vegetacao_nativa,
   FORMAT_NUMBER(area_remanescente_vegetacao_nativa / area_do_imovel * 100, 2) as percentual_area_nativa

FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`
"""

tempo_delta_5 = get_time_delta(query, "/home/jovyan/app/data/gold/delta/query_5")
tempo_delta_5

+-----------+--------------+----------------------------------+----------------------+
|codigo_ibge|area_do_imovel|area_remanescente_vegetacao_nativa|percentual_area_nativa|
+-----------+--------------+----------------------------------+----------------------+
|    3524709|        2.1643|                        0.16377169|                  7.57|
|    3507605|        1.4262|                               0.0|                  0.00|
|    3550605|        0.8601|                         0.5189845|                 60.34|
|    3541802|       46.5078|                               0.0|                  0.00|
|    3528809|       58.1119|                         0.7498494|                  1.29|
|    3510401|        1.1288|                        0.04275784|                  3.79|
|    3556701|        2.3052|                               0.0|                  0.00|
|    3537107|       229.792|                          69.41208|                 30.21|
|    3500303|        7.2557|               

25.969940662384033

## Consulta 6
Construa uma consulta que mostre a contagem de propriedades rurais por estado.

* AST – Assentamentos
* CAR – Cadastro Ambiental Rural
* CCU – Contrato de Concessão de Uso
* CDRU – Concessão de Direito Real de Uso
* DD – Diretoria de Desenvolvimento
* IRU – Imóveis Rurais
* MLC – Módulo Lote CAR
* PA – Projeto de Assentamento
* PCT – Povos e Comunidades Tradicionais
* PRA – Programa de Regularização Ambiental
* PNRA – Programa Nacional de Reforma Agrária
* SICAR – Sistema de Cadastro Ambiental Rural
* SR – Superintendência Regional
* TCMS – Termo de Compromisso de Manutenção de Sigilo
* TD – Título Definitivo
* TED – Termo de Execução Descentralizada
* URL – Uniform Resource Locator

In [10]:
query = """
SELECT
DISTINCT tipo_imovel_rural
                    
FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`
"""

spark.sql(query).show()

+-----------------+
|tipo_imovel_rural|
+-----------------+
|              AST|
|              IRU|
|              PCT|
+-----------------+



In [11]:
query = """
SELECT
    uf,
    COUNT(tipo_imovel_rural) AS count_prop_rural

FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`

WHERE tipo_imovel_rural = 'IRU'

GROUP BY uf

ORDER BY count_prop_rural DESC
"""

tempo_delta_6 = get_time_delta(query, "/home/jovyan/app/data/gold/delta/query_6")
tempo_delta_6

+---+----------------+
| uf|count_prop_rural|
+---+----------------+
| BA|         1011503|
| MG|          988921|
| RS|          606013|
| PR|          495650|
| SP|          406885|
| SC|          374768|
| PE|          338392|
| CE|          314168|
| PA|          278097|
| MA|          272010|
| PI|          254577|
| GO|          198524|
| PB|          170484|
| MT|          169775|
| RO|          147099|
| AL|          113222|
| ES|          108848|
| SE|           94203|
| RN|           88788|
| TO|           84990|
+---+----------------+
only showing top 20 rows



11.769421815872192

## Consulta 7
Veja qual é a maior propriedade entre todas e calcule a distância entre ela e Brasília. Utilize a coordenada de centródide da propriedade para calcular a distância entre ela e Brasília. Coordenadas de Brasília: -15.796943053171708, -47.891638482569476

In [24]:
query = """
SELECT
    max_area,
    latitude,
    longitude,
    FORMAT_NUMBER(
        SQRT(
            POWER(latitude - (-15.796943053171708), 2) + 
            POWER(longitude - (-47.891638482569476), 2) 
            )* 111, 2
        )  AS distancia_brasilia

FROM (

    SELECT 
        latitude, 
        longitude, 
        area_do_imovel AS max_area

    FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`

    ORDER BY area_do_imovel DESC

    LIMIT 1
)
"""

tempo_delta_7 = get_time_delta(query, "/home/jovyan/app/data/gold/delta/query_7")
tempo_delta_7

+---------+---------+---------+------------------+
| max_area| latitude|longitude|distancia_brasilia|
+---------+---------+---------+------------------+
|2420078.8|-5.475619|-68.88819|          2,596.98|
+---------+---------+---------+------------------+



10.904946327209473

## Consulta 8
Faça a média de área entre todas as propriedades. Calcule quantas propriedades por estado, estão acima da média.

In [12]:
query = """
WITH 
    avg_calc AS (
        SELECT
            AVG(area_do_imovel) AS avg_area

        FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`

        GROUP BY uf
        ), 

    filtro_estado AS (
        SELECT
            uf,
            area_do_imovel

        FROM delta.`/home/jovyan/app/data/silver/silver_distribuida_uf`, avg_calc

        WHERE area_do_imovel > avg_calc.avg_area
)

SELECT
    uf,
    COUNT(*) AS acima_media

FROM filtro_estado

GROUP BY uf

ORDER BY acima_media DESC

"""

tempo_delta_8 = get_time_delta(query, "/home/jovyan/app/data/gold/delta/query_8")
tempo_delta_8

+---+-----------+
| uf|acima_media|
+---+-----------+
| MG|    4659636|
| PA|    3040492|
| MT|    2423798|
| BA|    2301168|
| GO|    1991271|
| SP|    1942999|
| RS|    1930717|
| PR|    1668279|
| MA|    1628578|
| RO|    1337519|
| PI|    1263794|
| CE|    1043851|
| TO|     979080|
| MS|     957729|
| AM|     732345|
| SC|     729461|
| PE|     661795|
| AC|     492997|
| ES|     395320|
| PB|     385550|
+---+-----------+
only showing top 20 rows



19.08970046043396

## Tempo Total Delta Tables

In [25]:
total_delta = tempo_delta_1 + tempo_delta_2 + tempo_delta_3 + tempo_delta_4 + tempo_delta_5 + tempo_delta_6 + tempo_delta_7 + tempo_delta_8
total_delta

141.6745183467865

# Querys a partir da Hive Metastore

## Salvando Gold (Distribuída)

In [17]:
spark.sql("CREATE SCHEMA IF NOT EXISTS gold_from_delta")

DataFrame[]

In [19]:
df_test = spark.read.format("delta").load("/home/jovyan/app/data/silver/silver_distribuida_uf")

df_test.write.partitionBy("uf").saveAsTable("gold_from_delta.gold_distribuida_uf")

## Funções e Definições

In [28]:
def get_time_hive(query, table_name):
    start_time = time.time()

    table = spark.sql(query)

    (table.write.mode("overwrite")
                .saveAsTable(f"gold_from_delta.{table_name}"))
    
    spark.sql(f"""SELECT * FROM gold_from_delta.{table_name}""").show()

    end_time = time.time()

    tempo_query = end_time - start_time

    return tempo_query

## Consulta 1

Recupere a soma de área (em hectares) para todas as propriedades agrícolas que pertencem ao MS e MT. Ordene os resultados em ordem decrescente.

In [29]:
query = """
SELECT 
   uf,
   FORMAT_NUMBER(SUM(area_do_imovel), 3) as soma_area_hectares

FROM gold_from_delta.gold_distribuida_uf

WHERE uf IN ('MS', 'MT')

GROUP BY uf

ORDER BY soma_area_hectares DESC

"""

tempo_hive_1 = get_time_hive(query, "query_1")
tempo_hive_1

+---+------------------+
| uf|soma_area_hectares|
+---+------------------+
| MT|    84,587,089.218|
| MS|    36,321,882.660|
+---+------------------+



1.3467965126037598

## Consulta 2
Filtre todas as propriedades que pertecem a região sudeste. 

In [33]:
query = """
SELECT
   uf,
   codigo_ibge

FROM gold_from_delta.gold_distribuida_uf

"""

tempo_hive_2 = get_time_hive(query, "query_2")
tempo_hive_2

+---+-----------+
| uf|codigo_ibge|
+---+-----------+
| SC|    4212205|
| SC|    4201273|
| SC|    4207809|
| SC|    4214300|
| SC|    4212106|
| SC|    4202800|
| SC|    4212502|
| SC|    4208609|
| SC|    4204103|
| SC|    4208401|
| SC|    4203808|
| SC|    4203006|
| SC|    4216255|
| SC|    4213708|
| SC|    4210704|
| SC|    4209300|
| SC|    4217006|
| SC|    4202131|
| SC|    4211652|
| SC|    4204194|
+---+-----------+
only showing top 20 rows



5.564049243927002

## Consulta 3
Recupere todas as propriedades rurais que estão localizadas dentro de uma área geográfica específica delimitada por um polígono. Este polígono é descrito pelas seguintes coordenadas: POLYGON ((-53.5325072 -19.4632582, -51.0495971 -19.1625841, -51.3734501 -16.1924262, -53.8181518 -16.4010783, -53.5325072 -19.4632582))
Verificar se um centroide está dentro dos pontos informados (ou mais).

In [34]:
query = """
SELECT
    codigo_ibge,
    latitude,
    longitude,
    is_point_in_polygon(latitude, longitude) AS inside_polygon

FROM gold_from_delta.gold_distribuida_uf

WHERE is_point_in_polygon(latitude, longitude) = True
"""

tempo_hive_3 = get_time_hive(query, "query_3")
tempo_hive_3

+-----------+--------+---------+--------------+
|codigo_ibge|latitude|longitude|inside_polygon|
+-----------+--------+---------+--------------+
+-----------+--------+---------+--------------+



13.390249013900757

## Consulta 4
Calcule quantas propriedades foram cadastradas por ano. Apresente os resultados em ordem cronológica.

In [35]:
query = """
SELECT
   YEAR(data_inscricao) AS ano_inscricao,
   COUNT(*) AS qnt_ano

FROM gold_from_delta.gold_distribuida_uf

GROUP BY ano_inscricao

ORDER BY ano_inscricao
"""

tempo_hive_4 = get_time_hive(query, "query_4")
tempo_hive_4

+-------------+-------+
|ano_inscricao|qnt_ano|
+-------------+-------+
|         NULL|     12|
|         2013|    743|
|         2014| 227468|
|         2015|1240176|
|         2016|1828786|
|         2017| 794777|
|         2018| 724856|
|         2019| 746570|
|         2020| 463322|
|         2021| 443942|
|         2022| 368448|
+-------------+-------+



2.9008851051330566

## Consulta 5
Calcule o percentual médio de área remanescente de vegetação nativa em comparação a área total da propriedade

In [36]:
query = """
SELECT
   codigo_ibge,
   area_do_imovel,
   area_remanescente_vegetacao_nativa,
   FORMAT_NUMBER(area_remanescente_vegetacao_nativa / area_do_imovel * 100, 2) as percentual_area_nativa

FROM gold_from_delta.gold_distribuida_uf
"""

tempo_hive_5 = get_time_hive(query, "query_5")
tempo_hive_5

+-----------+--------------+----------------------------------+----------------------+
|codigo_ibge|area_do_imovel|area_remanescente_vegetacao_nativa|percentual_area_nativa|
+-----------+--------------+----------------------------------+----------------------+
|    4212205|       28.0898|                               0.0|                  0.00|
|    4201273|        5.0051|                               0.0|                  0.00|
|    4207809|     2306.3633|                          1770.341|                 76.76|
|    4214300|      101.7228|                          35.28556|                 34.69|
|    4212106|       12.9867|                         3.1989775|                 24.63|
|    4202800|       22.1912|                          2.443931|                 11.01|
|    4212502|        3.3022|                         1.2982384|                 39.31|
|    4208609|         4.846|                         1.7747418|                 36.62|
|    4204103|       24.8799|               

7.592971086502075

## Consulta 6
Construa uma consulta que mostre a contagem de propriedades rurais por estado.

In [37]:
query = """
SELECT
    uf,
    COUNT(tipo_imovel_rural) AS count_prop_rural

FROM gold_from_delta.gold_distribuida_uf

WHERE tipo_imovel_rural = 'IRU'

GROUP BY uf

ORDER BY count_prop_rural DESC
"""

tempo_hive_6 = get_time_hive(query, "query_6")
tempo_hive_6

+---+----------------+
| uf|count_prop_rural|
+---+----------------+
| BA|         1011503|
| MG|          988921|
| RS|          606013|
| PR|          495650|
| SP|          406885|
| SC|          374768|
| PE|          338392|
| CE|          314168|
| PA|          278097|
| MA|          272010|
| PI|          254577|
| GO|          198524|
| PB|          170484|
| MT|          169775|
| RO|          147099|
| AL|          113222|
| ES|          108848|
| SE|           94203|
| RN|           88788|
| TO|           84990|
+---+----------------+
only showing top 20 rows



2.4305200576782227

## Consulta 7
Veja qual é a maior propriedade entre todas e calcule a distância entre ela e Brasília. Utilize a coordenada de centródide da propriedade para calcular a distância entre ela e Brasília. Coordenadas de Brasília: -15.796943053171708, -47.891638482569476

In [38]:
query = """
SELECT
    max_area,
    latitude,
    longitude,
    FORMAT_NUMBER(
        SQRT(
            POWER(latitude - (-15.796943053171708), 2) + 
            POWER(longitude - (-47.891638482569476), 2) 
            )* 111, 2
        )  AS distancia_brasilia

FROM (

    SELECT 
        latitude, 
        longitude, 
        area_do_imovel AS max_area

    FROM gold_from_delta.gold_distribuida_uf

    ORDER BY area_do_imovel DESC

    LIMIT 1
)
"""

tempo_hive_7 = get_time_hive(query, "query_7")
tempo_hive_7

+---------+---------+---------+------------------+
| max_area| latitude|longitude|distancia_brasilia|
+---------+---------+---------+------------------+
|2420078.8|-5.475619|-68.88819|          2,596.98|
+---------+---------+---------+------------------+



2.6913340091705322

## Consulta 8
Faça a média de área entre todas as propriedades. Calcule quantas propriedades por estado, estão acima da média.

In [39]:
query = """
WITH 
    avg_calc AS (
        SELECT
            AVG(area_do_imovel) AS avg_area

        FROM gold_from_delta.gold_distribuida_uf

        GROUP BY uf
        ), 

    filtro_estado AS (
        SELECT
            uf,
            area_do_imovel

        FROM gold_from_delta.gold_distribuida_uf, avg_calc

        WHERE area_do_imovel > avg_calc.avg_area
)

SELECT
    uf,
    COUNT(*) AS acima_media

FROM filtro_estado

GROUP BY uf

ORDER BY acima_media DESC

"""

tempo_hive_8 = get_time_hive(query, "query_8")
tempo_hive_8

+---+-----------+
| uf|acima_media|
+---+-----------+
| MG|    4659636|
| PA|    3040492|
| MT|    2423798|
| BA|    2301168|
| GO|    1991271|
| SP|    1942999|
| RS|    1930717|
| PR|    1668279|
| MA|    1628578|
| RO|    1337519|
| PI|    1263794|
| CE|    1043851|
| TO|     979080|
| MS|     957729|
| AM|     732345|
| SC|     729461|
| PE|     661795|
| AC|     492997|
| ES|     395320|
| PB|     385550|
+---+-----------+
only showing top 20 rows



4.643881559371948

## Tempo Total Hive Metastore

In [40]:
total_hive = tempo_hive_1 + tempo_hive_2 + tempo_hive_3 + tempo_hive_4 + tempo_hive_5 + tempo_hive_6 + tempo_hive_7 + tempo_hive_8
total_hive

40.56068658828735

In [41]:
spark.sql("SHOW SCHEMAS").show()

+---------------+
|      namespace|
+---------------+
|        default|
|gold_from_delta|
+---------------+



In [42]:
spark.sql("SHOW TABLES IN gold_from_delta").show()

+---------------+-------------------+-----------+
|      namespace|          tableName|isTemporary|
+---------------+-------------------+-----------+
|gold_from_delta|gold_distribuida_uf|      false|
|gold_from_delta|            query_1|      false|
|gold_from_delta|            query_2|      false|
|gold_from_delta|            query_3|      false|
|gold_from_delta|            query_4|      false|
|gold_from_delta|            query_5|      false|
|gold_from_delta|            query_6|      false|
|gold_from_delta|            query_7|      false|
|gold_from_delta|            query_8|      false|
+---------------+-------------------+-----------+



# Querys a partir de Banco Centralizado (Professor)

In [ ]:
spark_mysql = (SparkSession.builder
               .getOrCreate())